## SQL Project with 3 datasets




### datasets


* Chicago Census Data


* Chicago Public Schools


* Chicago Crime Data






In [1]:
!pip install pandas
!pip install ipython-sql prettytable 

import prettytable

prettytable.DEFAULT = 'DEFAULT'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 134.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 156.4 MB/s eta 0:00:00


### Storing the datasets in database tables

To analyze the data using SQL, it first needs to be loaded into SQLite DB.

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**


In [ ]:
CHICAGO_CRIME_DATA = pd.read_csv("ChicagoCrimeData.csv")
CHICAGO_PUBLIC_SCHOOLS = pd.read_csv("ChicagoPublicSchools.csv")
CHICAGO_CENSUS_DATA = pd.read_csv("ChicagoCensusData.csv")

In [2]:
import pandas as pd
import sqlite3 as sql
conn = sql.connect('FinalDB.db')

In [4]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%sql sqlite:///FinalDB.db


In [ ]:
CHICAGO_CRIME_DATA.to_sql('crime_data', conn, if_exists='replace', index=False)
CHICAGO_PUBLIC_SCHOOLS.to_sql('public_schools', conn, if_exists='replace', index=False)
CHICAGO_CENSUS_DATA.to_sql('census_data', conn, if_exists='replace', index=False)

conn.close()





##### Total number of crimes recorded in the CRIME table.


In [7]:
%sql select count(*) as total_crimes from CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


total_crimes
533




##### Community area names and numbers with per capita income less than 11000.


In [36]:
%sql select community_area_name, community_area_number from CENSUS_DATA where per_capita_income <11000 

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0




##### All case numbers for crimes involving minors.(Children are not considered minors for the purposes of crime analysis) 


In [69]:
%%sql SELECT case_number FROM CHICAGO_CRIME_DATA WHERE description LIKE '%minor%'
   OR description LIKE '%juvenile%'
   OR description LIKE '%underage%'
   AND description NOT LIKE '%child%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408




##### Kind of crimes that were recorded at schools. (With no repetitions)


In [76]:
%sql SELECT distinct primary_type FROM CHICAGO_CRIME_DATA WHERE location_description LIKE '%school%' 

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION




##### Type of schools along with the average safety score for each type.


In [82]:
%sql SELECT "Elementary, Middle, or High School", AVG("Safety_Score") as average_safety_score FROM CHICAGO_PUBLIC_SCHOOLS GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0




##### 5 community areas with highest % of households below poverty line.


In [37]:
%sql select community_area_name from CENSUS_DATA order by percent_households_below_poverty desc limit 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale
Fuller Park
Englewood
North Lawndale
East Garfield Park




##### Community area most crime prone displaying the community area number only.


In [41]:
%sql SELECT community_area_number FROM CHICAGO_CRIME_DATA WHERE community_area_number IS NOT NULL GROUP BY community_area_number ORDER BY count(*) DESC LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0




#####  Community area with highest hardship index


In [42]:
%sql select community_area_name from CENSUS_DATA where hardship_index = (select max(hardship_index) from CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale




##### Using a sub-query to determine the Community Area Name with most number of crimes.


In [8]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);




 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
